In [1]:
import django_initializer
from telemetry.models import FastLap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx

import plotly.express as px
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots
import plotly.graph_objects as go

import pandas as pd

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()

2023-05-07 16:11:28,009 DEBUG Influx: Connected to http://telemetry.b4mad.racing/


In [2]:
def lap_fig(df):
    fig = go.Figure()

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Brake"],
        marker=dict(size=1),
        name="Brake",
        line=dict(color="red"),
    )

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Throttle"],
        marker=dict(size=1),
        name="Throttle",
        line=dict(color="green"),
    )
    return fig

In [7]:
kwargs = {
    "game__name": "iRacing",
    "track__name": "fuji nochicane",
    "car__name": "Ferrari 488 GT3 Evo 2020",
}

fast_laps = FastLap.objects.filter(**kwargs)

data_frames = []
for fast_lap in fast_laps:
    print(fast_lap)
    lap = fast_lap.laps.first()
    lap_df = influx.telemetry_for_laps([lap], measurement="fast_laps", bucket="fast_laps")[0]
    lap_df = analyzer.resample(lap_df, columns=["Brake", "Throttle"])
    data_frames.append(
        {
            "fast_lap": fast_lap,
            "df": lap_df,
        }
    )
    fig = lap_fig(lap_df)
    fig.update_layout(title=dict(text=f"{fast_lap}"))
    fig.show()

2023-05-07 16:13:51,093 INFO Processing iRacing - fuji nochicane - Ferrari 488 GT3 Evo 2020
2023-05-07 16:13:51,093 INFO   track.id 409 car.id 9
2023-05-07 16:13:51,093 INFO   session 1682107191 lap.id 37672 number 4
2023-05-07 16:13:51,094 INFO   length 4459 time 97.0107 valid True
2023-05-07 16:13:51,094 INFO   start 2023-04-21 22:04:53.148000+00:00 end 2023-04-21 22:06:30.157000+00:00


iRacing Ferrari 488 GT3 Evo 2020 fuji nochicane


In [11]:
df = data_frames[0]["df"]
df.head()

,DistanceRoundTrack,Brake,Throttle
3500,3501.02,0.197784,0.000000
1207,1207.94,0.099249,0.003884
4089,4090.04,0.000000,1.000000
1402,1402.94,0.000000,1.000000
1735,1735.96,0.000000,0.598977
